In [3]:
import os
import cv2 #pip install opencv-[ython]
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join

Welcome, today I am going to try and classify some images I found on kaggle to Identify landscape

If you wish to follow along you can download the data here

https://www.kaggle.com/datasets/utkarshsaxenadn/landscape-recognition-image-dataset-12k-images

## Section 1: Looking at Training Images

In [5]:
training_path = r'C:\Users\brunolopez\mldata\landscape_recognition\Landscape Classification\Landscape Classification\Training Data'
testing_path = r'C:\Users\brunolopez\mldata\landscape_recognition\Landscape Classification\Landscape Classification\Testing Data'
val_path = r'C:\Users\brunolopez\mldata\landscape_recognition\Landscape Classification\Landscape Classification\Validation Data'

In [ ]:
#Also a list of the file categories


I am going to add a few helper functions that will make this task easier


In [13]:
directories = os.listdir(training_path)
directories

['Coast', 'Desert', 'Forest', 'Glacier', 'Mountain']

A couple of helper functions

In [19]:
def combo_list_dir():

    '''
    Helper function:
    List all of the files and subdirectories 3 x 5 = 15 total
    
    '''

    all_dir = []

    list_paths = [training_path, testing_path, val_path]

    for i in list_paths:
        for j in directories:
            all_dir.append(i + '\\' + j)
            print(i + '\\' + j)

    return all_dir

def list_files(dir_list):
    '''
    List all of the individual images inside of the subdirectories
    
    '''

    all_files = []

    for subdir in dir_list:
        onlyfiles = [f for f in listdir(subdir) if isfile(join(subdir, f))]
        all_files.append(onlyfiles)
        print(onlyfiles)

    return all_files
    




In [20]:
test = combo_list_dir()

C:\Users\brunolopez\mldata\landscape_recognition\Landscape Classification\Landscape Classification\Training Data\Coast
C:\Users\brunolopez\mldata\landscape_recognition\Landscape Classification\Landscape Classification\Training Data\Desert
C:\Users\brunolopez\mldata\landscape_recognition\Landscape Classification\Landscape Classification\Training Data\Forest
C:\Users\brunolopez\mldata\landscape_recognition\Landscape Classification\Landscape Classification\Training Data\Glacier
C:\Users\brunolopez\mldata\landscape_recognition\Landscape Classification\Landscape Classification\Training Data\Mountain
C:\Users\brunolopez\mldata\landscape_recognition\Landscape Classification\Landscape Classification\Testing Data\Coast
C:\Users\brunolopez\mldata\landscape_recognition\Landscape Classification\Landscape Classification\Testing Data\Desert
C:\Users\brunolopez\mldata\landscape_recognition\Landscape Classification\Landscape Classification\Testing Data\Forest
C:\Users\brunolopez\mldata\landscape_recog